In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Credit Card Balance

In [2]:
ccb = pd.read_parquet('data/credit_card_balance.parquet')

In [4]:
ids_columns = ['SK_ID_PREV', 'SK_ID_CURR']
time_columns = ['MONTHS_BALANCE']
obj_columns = ['NAME_CONTRACT_STATUS']
num_columns = [ c for c in ccb.select_dtypes(include='number').columns if c not in ids_columns + time_columns ]

### Agregated data

In [50]:
agg_functions = ['min', 'max', 'sum', 'mean', 'median']
agg_dict = dict((k,agg_functions) for k in num_columns)
agg_dict[time_columns[0]] = ['min', 'max', 'nunique']
agg_dict[ids_columns[0]] = ['nunique']
agg_ccb = ccb.groupby('SK_ID_CURR').agg(agg_dict)
agg_ccb.columns = ['_'.join(col).strip() for col in agg_ccb.columns.values]

In [51]:
agg_ccb.head()

,AMT_BALANCE_min,AMT_BALANCE_max,AMT_BALANCE_sum,AMT_BALANCE_mean,AMT_BALANCE_median,AMT_CREDIT_LIMIT_ACTUAL_min,AMT_CREDIT_LIMIT_ACTUAL_max,AMT_CREDIT_LIMIT_ACTUAL_sum,AMT_CREDIT_LIMIT_ACTUAL_mean,AMT_CREDIT_LIMIT_ACTUAL_median,...,SK_DPD_median,SK_DPD_DEF_min,SK_DPD_DEF_max,SK_DPD_DEF_sum,SK_DPD_DEF_mean,SK_DPD_DEF_median,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_nunique,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,0.0,0.00,0.000,0.000000,0.0,270000,270000,1620000,270000.000000,270000.0,...,0.0,0,0,0,0.000000,0.0,-6,-1,6,1
100011,0.0,189000.00,4031676.225,54482.111149,0.0,90000,180000,12150000,164189.189189,180000.0,...,0.0,0,0,0,0.000000,0.0,-75,-2,74,1
100013,0.0,161420.22,1743352.245,18159.919219,0.0,45000,157500,12645000,131718.750000,157500.0,...,0.0,0,1,1,0.010417,0.0,-96,-1,96,1
100021,0.0,0.00,0.000,0.000000,0.0,675000,675000,11475000,675000.000000,675000.0,...,0.0,0,0,0,0.000000,0.0,-18,-2,17,1
100023,0.0,0.00,0.000,0.000000,0.0,45000,225000,1080000,135000.000000,135000.0,...,0.0,0,0,0,0.000000,0.0,-11,-4,8,1


### Linear Regression Slope

In [10]:
def model_slope(df, y):
    df = df[['MONTHS_BALANCE', y]].dropna()
    if df.shape[0] > 1:
        y = df[y].values
        X = df[['MONTHS_BALANCE']].values
        return round(LinearRegression().fit(X, y).coef_[0], 5)
    else:
        return np.nan


In [11]:
slope_list = []
for c in num_columns:
    slope = ccb.groupby('SK_ID_CURR').apply(model_slope, y = c )
    slope_list.append(slope)

In [20]:
agg_ccb_slope = pd.concat(slope_list, axis=1)
agg_ccb_slope.columns = num_columns
agg_ccb_slope = agg_ccb_slope.add_suffix('_slope')

In [48]:
agg_ccb_slope.head()

,AMT_BALANCE_slope,AMT_CREDIT_LIMIT_ACTUAL_slope,AMT_DRAWINGS_ATM_CURRENT_slope,AMT_DRAWINGS_CURRENT_slope,AMT_DRAWINGS_OTHER_CURRENT_slope,AMT_DRAWINGS_POS_CURRENT_slope,AMT_INST_MIN_REGULARITY_slope,AMT_PAYMENT_CURRENT_slope,AMT_PAYMENT_TOTAL_CURRENT_slope,AMT_RECEIVABLE_PRINCIPAL_slope,AMT_RECIVABLE_slope,AMT_TOTAL_RECEIVABLE_slope,CNT_DRAWINGS_ATM_CURRENT_slope,CNT_DRAWINGS_CURRENT_slope,CNT_DRAWINGS_OTHER_CURRENT_slope,CNT_DRAWINGS_POS_CURRENT_slope,CNT_INSTALMENT_MATURE_CUM_slope,SK_DPD_slope,SK_DPD_DEF_slope
SK_ID_CURR,,,,,,,,,,,,,,,,,,,
100006,-0.00000,-0.00000,NaN,-0.00000,NaN,NaN,-0.00000,NaN,-0.00000,-0.00000,-0.00000,-0.00000,NaN,-0.00000,NaN,NaN,-0.00000,-0.00000,-0.00000
100011,-2855.09506,-1056.94187,-194.59459,-194.59459,0.0,0.0,-182.25400,-165.59460,-176.79919,-2753.57653,-2854.77931,-2854.77931,-0.00432,-0.00432,0.0,0.0,0.41812,0.00000,0.00000
100013,-738.42671,-1242.20022,-340.44944,-257.26058,0.0,0.0,-71.18689,-270.24543,-267.56400,-703.60312,-736.99024,-736.99024,-0.01379,-0.01042,0.0,0.0,0.17351,-0.00028,-0.00028
100021,-0.00000,-0.00000,NaN,-0.00000,NaN,NaN,-0.00000,NaN,-0.00000,-0.00000,-0.00000,-0.00000,NaN,-0.00000,NaN,NaN,-0.00000,-0.00000,-0.00000
100023,-0.00000,34285.71429,NaN,-0.00000,NaN,NaN,-0.00000,NaN,-0.00000,-0.00000,-0.00000,-0.00000,NaN,-0.00000,NaN,NaN,-0.00000,-0.00000,-0.00000


### Contract Status

In [22]:
ccb.NAME_CONTRACT_STATUS.value_counts()

Active           3698436
Completed         128918
Signed             11058
Demand              1365
Sent proposal        513
Refused               17
Approved               5
Name: NAME_CONTRACT_STATUS, dtype: int64

In [33]:
status_series_list = []
for status in set(ccb.NAME_CONTRACT_STATUS):
    
    s = ccb.groupby('SK_ID_CURR').NAME_CONTRACT_STATUS.apply(lambda x: status in list(x)).astype('int')
    s.name = 'NAME_CONTRACT_STATUS_' + status
    status_series_list.append(s)

agg_ccb_status = pd.concat(status_series_list, axis= 1)

### Final DataFrame

In [52]:
agg_ccb_final = pd.concat([agg_ccb, agg_ccb_slope, agg_ccb_status], axis= 1)
agg_ccb_final = agg_ccb_final.add_prefix('CCB_')
agg_ccb_final = agg_ccb_final.reset_index()

In [53]:
agg_ccb_final.head()

,SK_ID_CURR,CCB_AMT_BALANCE_min,CCB_AMT_BALANCE_max,CCB_AMT_BALANCE_sum,CCB_AMT_BALANCE_mean,CCB_AMT_BALANCE_median,CCB_AMT_CREDIT_LIMIT_ACTUAL_min,CCB_AMT_CREDIT_LIMIT_ACTUAL_max,CCB_AMT_CREDIT_LIMIT_ACTUAL_sum,CCB_AMT_CREDIT_LIMIT_ACTUAL_mean,...,CCB_CNT_INSTALMENT_MATURE_CUM_slope,CCB_SK_DPD_slope,CCB_SK_DPD_DEF_slope,CCB_NAME_CONTRACT_STATUS_Active,CCB_NAME_CONTRACT_STATUS_Approved,CCB_NAME_CONTRACT_STATUS_Signed,CCB_NAME_CONTRACT_STATUS_Demand,CCB_NAME_CONTRACT_STATUS_Sent proposal,CCB_NAME_CONTRACT_STATUS_Completed,CCB_NAME_CONTRACT_STATUS_Refused
0,100006,0.0,0.00,0.000,0.000000,0.0,270000,270000,1620000,270000.000000,...,-0.00000,-0.00000,-0.00000,1,0,0,0,0,0,0
1,100011,0.0,189000.00,4031676.225,54482.111149,0.0,90000,180000,12150000,164189.189189,...,0.41812,0.00000,0.00000,1,0,0,0,0,0,0
2,100013,0.0,161420.22,1743352.245,18159.919219,0.0,45000,157500,12645000,131718.750000,...,0.17351,-0.00028,-0.00028,1,0,0,0,0,0,0
3,100021,0.0,0.00,0.000,0.000000,0.0,675000,675000,11475000,675000.000000,...,-0.00000,-0.00000,-0.00000,1,0,0,0,0,1,0
4,100023,0.0,0.00,0.000,0.000000,0.0,45000,225000,1080000,135000.000000,...,-0.00000,-0.00000,-0.00000,1,0,0,0,0,0,0


In [54]:
agg_ccb_final.to_pickle('data/ccb_agg_features.pkl')

## Installment Payments

In [ ]:
ip = pd.read_parquet('data/installments_payments.parquet')

### Calculate difference between intallment and payment

In [ ]:
ip['DAYS_DIFF'] =  ip['DAYS_INSTALMENT'] - ip['DAYS_ENTRY_PAYMENT']
ip['AMT_DIFF'] = ip['AMT_INSTALMENT'] - ip['AMT_PAYMENT']

### Agregated Data

In [ ]:
selected_columns = ['DAYS_DIFF', 'AMT_DIFF', 'NUM_INSTALMENT_NUMBER']

In [ ]:
agg_functions = ['min', 'max', 'sum', 'mean', 'median']
agg_dict = dict((k,agg_functions) for k in selected_columns)
agg_dict['SK_ID_PREV'] = ['nunique']
agg_ip = ip.groupby('SK_ID_CURR').agg(agg_dict)
agg_ip.columns = ['_'.join(col).strip() for col in agg_ip.columns.values]

In [ ]:
agg_ip = agg_ip.add_prefix('IP_')
agg_ip = agg_ip.reset_index()

### Final Dataset

In [ ]:
agg_ip.head()

,SK_ID_CURR,IP_DAYS_DIFF_min,IP_DAYS_DIFF_max,IP_DAYS_DIFF_sum,IP_DAYS_DIFF_mean,IP_DAYS_DIFF_median,IP_AMT_DIFF_min,IP_AMT_DIFF_max,IP_AMT_DIFF_sum,IP_AMT_DIFF_mean,IP_AMT_DIFF_median,IP_NUM_INSTALMENT_NUMBER_min,IP_NUM_INSTALMENT_NUMBER_max,IP_NUM_INSTALMENT_NUMBER_sum,IP_NUM_INSTALMENT_NUMBER_mean,IP_NUM_INSTALMENT_NUMBER_median,IP_SK_ID_PREV_nunique
0,100001,-11.0,36.0,51.0,7.285714,6.0,0.0,0.0,0.0,0.0,0.0,1,4,19,2.714286,3.0,2
1,100002,12.0,31.0,388.0,20.421053,19.0,0.0,0.0,0.0,0.0,0.0,1,19,190,10.000000,10.0,1
2,100003,1.0,14.0,179.0,7.160000,6.0,0.0,0.0,0.0,0.0,0.0,1,12,127,5.080000,5.0,3
3,100004,3.0,11.0,23.0,7.666667,9.0,0.0,0.0,0.0,0.0,0.0,1,3,6,2.000000,2.0,1
4,100005,-1.0,37.0,212.0,23.555556,29.0,0.0,0.0,0.0,0.0,0.0,1,9,45,5.000000,5.0,1


In [ ]:
agg_ip.to_pickle('data/ip_agg_features.pkl')